In [ ]:
!pip install datasets transformers torch
!pip install pandas
import pandas as pd
import json
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

In [ ]:
import json
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

In [ ]:
# Load dataset
file_path = "/content/CancerQA.csv"
df = pd.read_csv(file_path)

# Prepare conversation pairs
conversations = []
for entry in df.to_dict(orient='records'):
    # Access the correct column names 'Question' and 'Answer'
    user_input = entry["Question"]
    bot_response = entry["Answer"]
    conversations.append(f"User: {user_input}\nBot: {bot_response}")

In [ ]:
# Load GPT-2 tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token explicitly
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Tokenize conversation data
tokenized_data = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized data into a Hugging Face Dataset
dataset = Dataset.from_dict({
    "input_ids": tokenized_data["input_ids"],
    "attention_mask": tokenized_data["attention_mask"]
})

# Print a sample to verify everything works
print(dataset[0])

{'input_ids': [12982, 25, 1867, 318, 357, 533, 8, 8504, 12, 18712, 12440, 39337, 15523, 5633, 198, 20630, 25, 7383, 11045, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 532, 8504, 12, 17470, 2685, 12317, 4890, 318, 257, 4369, 287, 543, 6428, 25114, 357, 48870, 8, 4778, 1296, 287, 262, 21379, 286, 262, 12317, 13, 220, 220, 220, 220, 532, 1318, 389, 1811, 3858, 286, 1729, 12, 17470, 2685, 12317, 4890, 13, 220, 220, 220, 532, 39801, 318, 262, 1688, 2526, 5766, 329, 1729, 12, 17470, 2685, 12317, 4890, 13, 220, 220, 220, 532, 37027, 286, 1729, 12, 17470, 2685, 12317, 4890, 2291, 257, 22094, 326, 1595, 470, 467, 1497, 290, 1790, 1108, 286, 8033, 13, 220, 220, 220, 532, 30307, 326, 10716, 262, 21726, 389, 973, 284, 4886, 357, 19796, 828, 37489, 11, 290, 3800, 1729, 12, 17470, 2685, 12317, 4890, 13, 220, 220, 220, 532, 16272, 5087, 2689, 1172, 31707, 357, 39486, 286, 7628, 8, 290, 3513, 3689, 13, 220, 220, 220, 532, 1114, 749, 3871, 351, 17

In [ ]:
training_args = TrainingArguments(
    output_dir="./chatbot_model",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.098200
1000,0.759700


TrainOutput(global_step=1095, training_loss=0.9106380932951627, metrics={'train_runtime': 799.8374, 'train_samples_per_second': 2.734, 'train_steps_per_second': 1.369, 'total_flos': 1142891347968000.0, 'train_loss': 0.9106380932951627, 'epoch': 3.0})

In [ ]:
def chatbot_response(user_input):
    # Define device here or before calling the function
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    input_text = f"User: {user_input}\nBot:"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    # Generate the response
    output = model.generate(input_ids, max_length=50, pad_token_id=tokenizer.eos_token_id)

    # Decode the response
    response = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    print("Bot:", chatbot_response(user_input))

You: What is (are) Non-Small Cell Lung Cancer ?


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot:  Key Points
                    - Non-small cell lung cancer is a disease in
You: What are the symptoms of Non-Small Cell Lun
Bot:  Signs of non-small cell lung include fever, weight loss, and tiredness. These and other signs and symptoms may be caused by non-small cell lung or by other
You: exit


In [ ]:
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2")
print("Fine-tuning complete! Model saved.")

Fine-tuning complete! Model saved.


In [34]:
from google.colab import files
import shutil

shutil.make_archive("fine_tuned_gpt2", 'zip', "/content/fine_tuned_gpt2")
files.download("fine_tuned_gpt2.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r chatbot_model.zip ./chatbot_model

  adding: chatbot_model/ (stored 0%)
  adding: chatbot_model/merges.txt (deflated 53%)
  adding: chatbot_model/tokenizer_config.json (deflated 54%)
  adding: chatbot_model/tokenizer.json (deflated 82%)
  adding: chatbot_model/checkpoint-1095/ (stored 0%)
  adding: chatbot_model/checkpoint-1095/training_args.bin (deflated 52%)
  adding: chatbot_model/checkpoint-1095/config.json (deflated 51%)
  adding: chatbot_model/checkpoint-1095/model.safetensors (deflated 7%)
  adding: chatbot_model/checkpoint-1095/rng_state.pth (deflated 25%)
  adding: chatbot_model/checkpoint-1095/generation_config.json (deflated 24%)
  adding: chatbot_model/checkpoint-1095/scheduler.pt (deflated 56%)
  adding: chatbot_model/checkpoint-1095/trainer_state.json (deflated 56%)
  adding: chatbot_model/checkpoint-1095/optimizer.pt (deflated 8%)
  adding: chatbot_model/config.json (deflated 51%)
  adding: chatbot_model/model.safetensors (deflated 7%)
  adding: chatbot_model/special_tokens_map.json (deflated 60%)
  addin

In [ ]:
from google.colab import files
files.download("/content/chatbot_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/my_model /content/drive/MyDrive/

Mounted at /content/drive
cp: cannot stat '/content/my_model': No such file or directory


In [ ]:
from google.colab import files
files.download("/content/chatbot_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>